## COVID19- Argentina


In [ ]:
#hide
import os
import numpy as np
import pandas as pd
import glob
import locale
import altair as alt
from altair import datum
from jinja2 import Template

#
from IPython.display import HTML
# Set Language
_=locale.setlocale(locale.LC_TIME,'es_ES.utf8')

In [ ]:
# helpers
def cat_df(data_src):
    lst_=glob.glob(data_src)
    lst_.sort()
    lst_df_=[]
    for l in lst_:
        df_=pd.read_csv(l,parse_dates=[0])
        lst_df_.append(df_)
    df_cat_=pd.concat(lst_df_,axis=0,)
    df_cat_['fecha']=pd.to_datetime(df_cat_['fecha'].dt.strftime('%Y-%m-%d'))
    # Group all with "FIX"
    df_cat_=df_cat_.groupby(['provincia','fecha'])[['confirmados','muertes','recuperados']].sum().reset_index()
    return df_cat_
#
provDict={'Ciudad Autónoma de Buenos Aire':'CABA'}


In [ ]:
# CSV sources
data_src_provincia = '../../data/filled/*provincia*.csv'
data_src_argentina = '../../data/filled/*argentina*.csv'
data_prod = '../../products'
data_includes_ = '../_includes'
os.makedirs(data_prod, exist_ok=True)
os.makedirs(data_includes_, exist_ok=True)
#
df_ = pd.concat([cat_df(data_src_provincia), cat_df(data_src_argentina)])
df_['provincia'] = df_['provincia'].apply(
    lambda x: provDict[x] if x in provDict else x)
df_.head()

In [ ]:
# make cumsum on df_
df_cumsum_=df_.groupby(['provincia'])[['confirmados','muertes','recuperados']].cumsum().add_prefix('cumsum_')
# concat dfs
df_cat=pd.concat([df_,df_cumsum_],axis=1)

In [ ]:
df_cat

In [ ]:
#
source=df_cat
circ=alt.Chart(source).mark_circle(color='orange').encode(
    x=alt.X('fecha:T',axis=alt.Axis(title='Fecha'),),
    y=alt.Y('provincia',axis=alt.Axis(title='Provincias/Nación')),
    size='confirmados',
    tooltip=['confirmados', 'fecha','provincia','muertes']
).properties(width=600, height=600).interactive()


In [ ]:
circ

In [ ]:
#
pts = alt.selection(type="single", encodings=['x'])
rect=alt.Chart(source).mark_rect().encode(
    alt.Y('provincia:O', title='Provincias/Nacion'),
    alt.X('monthdate(fecha):O', title='Fecha'),
    alt.Color('confirmados:Q', title='Casos Confirmados',),
tooltip=['confirmados', 'fecha','provincia','muertes','recuperados']
).properties(width=600, height=600).interactive()
rect

In [ ]:
from scipy import stats
df_reg=source[source.provincia=='Argentina_Nacion'].copy()
#0 based
dayFit=np.arange(6,15,1)
deltaT=dayFit[1]-dayFit[0]
y=df_reg.iloc[dayFit]['cumsum_confirmados'].values
x=dayFit
#
slope, intercept,_,_,_=stats.linregress(x,np.log(y))
#
fitted=np.exp(intercept)*np.exp(slope)**np.arange(0,df_reg.shape[0],1)
#
df_reg['fitted']=np.nan
df_reg['fitted']=fitted
df_reg['fitted_flag']='extrapolado'
df_reg.loc[dayFit,'fitted_flag']='ajustado'
df_reg['daily_expected']=df_reg['fitted'].diff().fillna(0)

In [ ]:
pts = alt.selection(type="single", encodings=['x'],empty='none')
#
points = alt.Chart(source).mark_circle(color='orange').encode(
    alt.X('fecha:T',title='Fecha'),
    alt.Y('cumsum_confirmados:Q',scale=alt.Scale(type='log'),title='#Confirmados Totales en Argentina (log scale)'),
    size=alt.Size('confirmados',title='#Confirmados(dia)'),
    color=alt.condition(pts, 'confirmados:Q', alt.value('orange'),legend=None),
    tooltip=['confirmados', 'fecha','provincia','muertes','recuperados']
).properties(        width=550,
        height=400).transform_filter((datum.provincia == 'Argentina_Nacion')).add_selection(pts)

tooltip_bars=[alt.Tooltip('cumsum_confirmados:Q', title='#Confirmados Totales'),alt.Tooltip('fecha:T', title='Fecha'),alt.Tooltip('provincia:N', title='Provincia')]

bars = alt.Chart(source).mark_bar().encode(
       alt.X('provincia:N',axis=alt.Axis(minExtent=100),title='Provincia de Residencia'),
        alt.Y('cumsum_confirmados:Q',title='Confirmados x Provincia'),
        color=alt.Color('cumsum_confirmados:Q',title='Confirmados x Provincia',legend=None),
        tooltip=tooltip_bars).properties(  
    width=550,
        height=220
    ).transform_filter(datum.provincia != 'Argentina_Nacion').transform_filter((pts)).interactive()

texts = alt.Chart(source).mark_text(dy=0, size=20).encode(
    text='fecha:T'
).transform_filter(
    pts
)

#Regression
reg= alt.Chart(df_reg).mark_line(opacity=0.2,point=True).encode(
    alt.X('fecha:T',title='Fecha'),
    alt.Y('fitted:Q',scale=alt.Scale(type='log'),title='#Confirmados Totales en Argentina (log scale)'),
    color=alt.Color('fitted_flag',
                   scale=alt.Scale(
            domain=['extrapolado', 'ajustado'],
            range=['red', 'green']),title='#Confirmados(t)=a*b^t'),
    tooltip=['daily_expected'], 
).properties(        width=550,
        height=400).interactive()


In [ ]:
alt.vconcat(points+reg, bars+texts)